# LAB 2 Andrea Mirenda s327644

The way to install any library using a linux terminal aproach is the following:

Import with predefined python environment

In [266]:
import pandas as pd
from dataclasses import dataclass
from collections import Counter
import random
import math
import numpy as np
from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate
from itertools import combinations
from tqdm.auto import tqdm
import geopy.distance
from geopy.distance import geodesic
import networkx as nx
from collections import deque

# data import
We use pandas to import and use the data about the cities.
Each city has the coordinates (x and y) which allows us to compute the Euclidean distance between 2 cities.

In [267]:
cities = pd.read_csv('cities/italy.csv', header=None, names=['City', 'x', 'y'])
cities

,City,x,y
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34
5,Bolzano,46.50,11.35
6,Brescia,45.55,10.22
7,Cagliari,39.22,9.10
8,Catania,37.50,15.08
9,Ferrara,44.84,11.61


In [268]:
DIST_MATRIX = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.x, c1.y), (c2.x, c2.y)
    ).km
num_cities = len(cities)

In [269]:
def find_closest(segments, city, visited):
    # Filtra i segmenti che contengono 'city' e hanno un'altra città non in 'visited'
    candidates = [
        (pair, distance) for pair, distance in segments
        if city in pair and (other_city := (pair - {city}).pop()) not in visited
    ]
    closest_segment = min(candidates, key=lambda x: x[1])
    closest_city = (closest_segment[0] - {city}).pop()
    return (int(city), closest_city)

In [270]:
def swap_mutation(solution, strength=0.5):
    while random.random()<strength:
        index = random.randint(1, len(solution) - 2) #not the first nor the last one
        index2=index #should be higher
        while index2<=index:
            index2 = random.randint(0, len(solution) - 1)
        # now we have the 2 indeces to use.
        selected_edge1 = solution[index]
        selected_edge2 = solution[index2]
        solution[index] = selected_edge2
        solution[index2] = selected_edge1
    #print("soluzione finale:", fitness(solution))
    return solution

In [271]:
def inversion_mutation(solution):
    # choose the index:
    index, index2 = np.random.randint(0, len(solution) - 1, size=2) #not the first nor the last one
    if index<index2:
        for _ in range(index, index2):
            sol=solution[index]
            solution[index]=solution[index+1]
            solution[index+1]=sol
    else:
        for _ in range(index2, index):
            sol=solution[index2]
            solution[index2]=solution[index2+1]
            solution[index2+1]=sol
    return solution

In [272]:
@dataclass
class Individual:
    genome: list
    fitness  = None

In [273]:
def cyclic(solution):
    edges = []
    for node in range(len(solution)-2):
        edges.append({solution[node], solution[node+1]})
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return 1
    except:
        return 0

In [274]:
def num_cyclic(solution):
    edges = []
    for node in range(len(solution)-2):
        edges.append({solution[node], solution[node+1]})
    G = nx.Graph()
    G.add_edges_from(edges)
    cycles = nx.cycle_basis(G)
    return len(cycles)

In [275]:
def greedy_sol(city, segments):
    solution = []
    solution.append(city)
    visited = []
    visited.append(int(city))
    while len(visited)<len(cities):
        _, c1 = find_closest(segments, city, visited)
        solution.append(c1)
        visited.append(c1)
        city=c1
    solution.append(solution[0])
    
    return solution

In [276]:
def fitness(solution):
    tot_dist=0
    for node in range(len(solution)-1):
        tot_dist -= DIST_MATRIX[solution[node], solution[node+1]]
    # edges = []
    # for node in range(len(solution)-1):
    #     edges.append({solution[node], solution[node+1]})
    #print("gli archi sono ", edges)
    return (1-num_cyclic(solution),len(solution), tot_dist)
    #return (len(solution), tot_dist)

In [277]:
def parent_selection(population):
    candidates = sorted(np.random.choice(population, 10), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


def inver_over_crossover(parent1, parent2):
    # Copia del primo genitore, su cui applicheremo le modifiche
    child = parent1[:]
    
    # Selezione di un punto di crossover casuale
    crossover_point = random.randint(1, len(parent1) - 2)
    
    # Segmento dal primo genitore (prima del punto di crossover)
    first_segment = parent1[:crossover_point]
    
    # Segmento dal secondo genitore (copia il collegamento per il resto)
    second_segment = []
    for city in parent2:
        if city not in first_segment:
            second_segment.append(city)
    
    # Combinazione dei segmenti per creare il figlio
    child = first_segment + second_segment
    
    return child

In [ ]:
POPULATION_SIZE = 10
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]

population = []
for i in range(POPULATION_SIZE):
    sol = greedy_sol(random.randint(0, num_cities), segments)
    population.append(Individual(sol))
#population = [Individual(greedy_sol(random.randint(0, num_cities), segments)) for _ in range(POPULATION_SIZE)]

for i in population:
    i.fitness = fitness(i.genome)
print("best solution before EA: ", population[0].fitness)

OFFSPRING_SIZE = 20
MAX_GENERATIONS=25_000


for g in range(MAX_GENERATIONS):
    offspring = list()
    
    for _ in range (OFFSPRING_SIZE):
        if np.random.random()<.7:#mutation probability:
            p=parent_selection(population)
            o=swap_mutation(p.genome)
        else:
            i1 = parent_selection(population)
            i2 = parent_selection(population)
            o = inver_over_crossover(i1.genome, i2.genome)
    offspring.append(Individual((o)))

for i in offspring:
    i.fitness = fitness(i.genome)

population.extend(offspring)
# for i in range(len(population)):
#     tmp = population[i].genome
#     population[i].genome.append(tmp[0])

#esclude the solution unfeasable:m
# new_population=population.copy()
# for i in range(len(population)):
#     if population[i].fitness[0]==0:
#         new_population.pop(i)
# population=new_population

population.sort(key=lambda i: i.fitness, reverse=True)
population = population[:POPULATION_SIZE]
print("best solution after EA: ", population[0].fitness)

best solution before EA:  (1, 47, np.float64(-5015.50007325793))
